# 事前学習済みモデルをファインチューニングする
事前学習済みモデルを使用することには大きな利点がある。計算コストを削減できるだけでなく、ゼロからトレーニングを行わなくても最新のモデルを利用できるからだ。

事前学習済みモデルを使用する場合、タスクに特化したデータセットでトレーニングを行う。これをファインチューニングと呼び、とても強力なトレーニング手法だ。このチュートリアルでは、お好みのディープラーニングフレームワークを使って事前学習済みモデルをファインチューニングする方法を学ぶ。[参考](https://huggingface.co/docs/transformers/v4.46.3/ja/training)


In [1]:
from datasets import load_dataset

# https://huggingface.co/datasets/Yelp/yelp_review_full
dataset = load_dataset("yelp_review_full")
dataset["train"][:2]

{'label': [4, 1],
 'text': ["dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
  "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have offi

In [2]:
small_train_dataset = dataset["train"].select(range(1000))
small_eval_dataset = dataset["test"].select(range(400))

In [3]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
small_train_dataset[0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 'input_ids': [101,
  173,
  1197,
  119,
  2284,
  2953,
  3272,
  1917,
  178,
  1440,
  1111,
  1107,
  170,
  1704,
  22351,
  119,
  1119,
  112,
  188,
  3505,
  1105,
  3123,
  1106,
  2037,
  1106,
  1443,
  1217,
  10063,
  4404,
  132,
  1119,
  112,
  188,
  1579,
  1113,
  1159,
  1107,
  3195,
  1117,
  4420,
  132,
  1119,
  112,
  188,
  6559,
  1114,
  170,
  1499,
  118,
  23555,
  2704,
  113,
  183,
  9379,
  114,
  1

In [5]:
small_train_dataset = small_train_dataset.remove_columns(["text"])
small_eval_dataset = small_eval_dataset.remove_columns(["text"])
small_train_dataset = small_train_dataset.rename_column("label", "labels")
small_eval_dataset = small_eval_dataset.rename_column("label", "labels")
small_train_dataset.set_format("torch")
small_eval_dataset.set_format("torch")

In [6]:
from transformers import AutoModelForSequenceClassification
# https://github.com/huggingface/transformers/blob/7ae6f070044b0171a71f3269613bf02fd9fca6f2/src/transformers/models/bert/modeling_bert.py#L1564-L1575
# このリンクはBertForSequenceClassification のソースコード、num_labelsは出力の数、今回は5つのラベルを出力する
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [11]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [12]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [13]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")  # macOS
elif torch.cuda.is_available():
    device = torch.device("cuda")  # GPU
else:
    device = torch.device("cpu")  

model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
from tqdm.auto import tqdm
import torch
import evaluate

# 精度指標を読み込む
metric = evaluate.load("accuracy")

# 最高精度と最適モデルの状態を初期化
best_accuracy = 0.0
best_model_state = None

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # バッチデータをGPU/CPUデバイスに転送
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)# モデルに入力してforward計算を実行
        loss = outputs.loss # 損失を計算
        loss.backward() # 勾配を計算（バックプロパゲーション）

        optimizer.step() # パラメータを更新
        lr_scheduler.step() # 学習率を更新
        optimizer.zero_grad() # 勾配をゼロに初期化（次のバッチの準備）
        progress_bar.update(1)

    # 各エポックの終わりに評価を実行
    model.eval()  # 評価モードに設定
    metric.reset()  # 評価指標をリセット

    with torch.no_grad():  # 勾配計算を無効化, inference時にメモリを節約
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            metric.add_batch(predictions=predictions, references=batch["labels"])

    eval_results = metric.compute()  # 精度を計算
    eval_accuracy = eval_results["accuracy"]
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {eval_accuracy}")

    # 現在のモデルの検証精度が高い場合、モデルを保存
    if eval_accuracy > best_accuracy:
        best_accuracy = eval_accuracy
        best_model_state = model.state_dict()  # 最良モデルの状態を保存

        # 最良モデルを保存
        torch.save(best_model_state, "best_model.pth")
        print(f"Saved best model at epoch {epoch + 1} with accuracy {eval_accuracy}")

    model.train()  # トレーニングモードに戻す

  0%|          | 0/375 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def inference_pipeline(model, text, tokenizer, device, max_length=128):

    # model.eval()を呼び出すことで、モデルを推論モードに設定
    model.eval()
    
    # トークナイザーを使用してテキストをエンコード
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    encoding = {k: v.to(device) for k, v in encoding.items()}
    
    # 推論モード
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
        
        # ソフトマックス関数を使用して確率に変換
        probabilities = F.softmax(logits, dim=-1)
        predicted_label = torch.argmax(logits, dim=-1)
        
        # 結果をCPUに移動して返す
        probabilities = probabilities.cpu().numpy()[0]
        predicted_label = predicted_label.cpu().numpy()[0]
    
    return predicted_label, probabilities

In [ ]:
text_samples = [
    "text1",
    "text2"
]

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
model_path = "best_model.pth"


model.load_state_dict(torch.load(model_path))
model.to(device)
    

for text in text_samples:
    label, probs = inference_pipeline(model, text, tokenizer, device)
    print(f"\n文: {text}")
    print(f"モデルの予測結果: {label}")
    print(f"確率: {probs}")


# [参考](https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification)